Fahe News Classifier Using Bidirectional Lstm

Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fake News Classifier using LSTM/fake_train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.shape

(20800, 5)

In [7]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
#drop nan values
df=df.dropna()

In [11]:
df.shape

(18285, 5)

In [12]:
#Independent features
X=df.drop('label',axis=1)


In [13]:
#dependent features
y=df['label']


In [14]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [15]:
X.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [17]:
import tensorflow as tf
tf.__version__ #if version greater than2.0 will use tf.keras if below use keras

'2.8.0'

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [19]:
# vocabulary size
voc_size=5000

Onehot Representation

In [20]:
messages=X.copy()

In [23]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [24]:
messages.reset_index(inplace=True)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Data Preprocessing

In [27]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  
  review =[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review =' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [29]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

[[4292, 930, 682, 4641, 977, 1418, 4049, 3740, 123, 4644],
 [667, 388, 2394, 3221, 4096, 3231, 135],
 [244, 1390, 3134, 1966],
 [3884, 3272, 1403, 1010, 311, 2661],
 [1225, 4096, 4871, 3968, 2817, 3063, 4096, 4746, 1389, 75],
 [1320,
  181,
  3020,
  3824,
  3576,
  3067,
  797,
  2653,
  699,
  4970,
  848,
  4650,
  2158,
  117,
  135],
 [3474, 3930, 2415, 729, 3425, 3093, 4781, 4363, 3659, 2836, 4559],
 [3801, 3141, 2610, 841, 4682, 332, 3067, 2906, 3659, 2836, 4559],
 [864, 546, 2722, 4654, 1070, 2810, 3786, 3560, 3067, 3137],
 [3664, 246, 3250, 2875, 1483, 3249, 2151, 1428],
 [728, 2504, 1523, 1983, 1287, 1144, 3335, 99, 1377, 1251, 252],
 [1010, 1162, 977, 2810, 3067, 4682],
 [719, 232, 3934, 606, 3998, 4097, 4790, 4948, 2871],
 [4474, 4175, 808, 3957, 4860, 4359, 1049, 3659, 2836, 4559],
 [4699, 2072, 1525, 947, 2875, 3659, 2836, 4559],
 [4045, 4101, 4030, 4867, 4773, 4306, 4497, 635, 3676, 1430],
 [3422, 293, 388],
 [215, 3262, 3205, 2048, 3067, 3900, 2135, 135],
 [1179, 4400, 

Embedding Representation

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3740  123 4644]
 [   0    0    0 ... 4096 3231  135]
 [   0    0    0 ... 1390 3134 1966]
 ...
 [   0    0    0 ... 3659 2836 4559]
 [   0    0    0 ... 4855 3882 4827]
 [   0    0    0 ... 4421 4273  746]]


In [31]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4292,
        930,  682, 4641,  977, 1418, 4049, 3740,  123, 4644], dtype=int32)

Creating model

In [32]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


 using Dropout method and biodirectional LSTM

In [51]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
len(embedded_docs),y.shape

(18285, (18285,))

In [53]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [54]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [56]:
model1.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 25ms/step - loss: 0.3017 - accuracy: 0.8668 - val_loss: 0.1968 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1302 - accuracy: 0.9513 - val_loss: 0.2149 - val_accuracy: 0.9153
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0908 - accuracy: 0.9672 - val_loss: 0.2581 - val_accuracy: 0.9128
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0682 - accuracy: 0.9754 - val_loss: 0.2867 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0430 - accuracy: 0.9856 - val_loss: 0.3824 - val_accuracy: 0.9094
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0298 - accuracy: 0.9900 - val_loss: 0.4275 - val_accuracy: 0.9077
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.4756 - val_accuracy: 0.906

Performance Metrics And Accuracy

In [57]:
p_pred=model1.predict(X_test)
p_pred = p_pred.flatten()
print(p_pred.round(2))

[1.   0.   0.   ... 0.01 1.   1.  ]


In [58]:
y_pred = np.where(p_pred > 0.5, 1, 0)
print(y_pred)

[1 0 0 ... 0 1 1]


In [59]:
from sklearn.metrics import confusion_matrix

In [64]:
print(confusion_matrix(y_test,y_pred))

[[3098  321]
 [ 224 2392]]


In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9096934548467275

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

